Notebook outlining what functions from nltk will be useful for pre-processing the text and training the classification algorithm.

Expects a CSV file in the current directory containing a release's date, text, and classification (pos or neg) in each row.

In [17]:
import pandas as pd
import nltk
import re

In [7]:
#release_df = pd.read_csv('textsdf.csv', index_col=0)


In [15]:
release_df['year'] = release_df['date'].apply(lambda x: int(x[-4:]))

In [29]:
release_df['month'] = release_df['date'].apply(lambda x: re.findall(r'(\w+) \d', x)[0])

In [30]:
release_df[:5]

,date,release_text,year,month
0,"January 31, 2018",Information received since the Federal Open Ma...,2018,January
1,"February 01, 2017",Information received since the Federal Open Ma...,2017,February
2,"March 15, 2017",Information received since the Federal Open Ma...,2017,March
3,"May 03, 2017",Information received since the Federal Open Ma...,2017,May
4,"June 14, 2017",Information received since the Federal Open Ma...,2017,June


In [32]:
release_df.to_csv('textsdf.csv')

In [ ]:
#release_df['tokenized'] = release_df['text']